In [12]:
import zipfile, os, subprocess, contextlib

def update(public, lvws):
    """ Updates the labview webservice file 'lvws' using the 
    public content folder 'public'. No sanity checking - 
    just don't fuck up :)
    """
    with zipfile.ZipFile(lvws, 'r') as old:
        with zipfile.ZipFile(lvws+".new", 'w') as new:
            for file in old.namelist():
                if not file.lower().startswith("public/"):
                    new.writestr(file, old.read(file))
            for d, _, files in os.walk(public):
                for file in files:
                    ffp = os.path.join(d,file)
                    rel = os.path.relpath(ffp, public)
                    out = os.path.join("Public", rel)
                    with open(ffp, 'rb') as f:
                        new.writestr(out, f.read())
    
    bak = lvws+".bak"
    try:
        os.remove(bak)
    except FileNotFoundError:
        pass
    os.rename(lvws, bak)
    os.rename(lvws+".new", lvws)
    
@contextlib.contextmanager
def tchdir(new):
    old = os.path.abspath(".")
    os.chdir(new)
    try:
        yield
    finally:
        os.chdir(old)

def build(bdir):
    # move to the builds dir and run ant
    # throw an error if the build fails
    with tchdir(bdir):
        subprocess.check_call("ant", shell=True)
    

In [13]:
sweng = os.path.expanduser("~\\documents\\pbs\\software engineering")

ws = os.path.join(sweng, "webservice.lvws")
repo = os.path.join(sweng, "uidev_dualsens")

public = os.path.join(repo, "public")
builds = os.path.join(repo, "build")

build(builds)
update(public, ws)